In [ ]:
import importlib
import ifera
import torch

importlib.reload(ifera)

cm = ifera.ConfigManager()
# instrument = config.get_config('IBKR', 'NIY:1m')

# ifera.check_s3_file_exists(S3_BUCKET, ifera.make_s3_key(instrument, True))

# df_raw = ifera.load_data(raw=True, instrument=instrument, zipfile=True)

# df = ifera.load_data(raw=False, instrument=instrument, zipfile=True, reset=True)

# instrument = config.get_config('IBKR', 'HE:1m')
# t_old = ifera.load_data_tensor(instrument=instrument, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)
# t_new = ifera.load_data_tensor(instrument=instrument, reset=True, device=torch.device('cuda:0'), dtype=torch.float32)

# t_new.isclose(t_old).all()

# instrument = config.get_config('IBKR', 'NN:1m')
# t_nn = ifera.load_data_tensor(instrument=instrument, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)

iconfig = cm.get_config('IBKR', 'GF:1m')
#t = ifera.load_data_tensor(instrument=instrument, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)

instrument2 = cm.get_config('IBKR', 'GF:1m')
instrument3 = iconfig.model_copy()

iconfig.__eq__(instrument2)

In [ ]:


import pandas as pd

all_steps = pd.timedelta_range(start=pd.Timedelta(0), end=pd.Timedelta(days=1), freq=iconfig.time_step)
# Filter out steps that are greater than or equal to instrument end time
all_steps[all_steps < iconfig.trading_end - iconfig.trading_start][-1]


    


In [ ]:
import ifera

cm = ifera.ConfigManager()
brokerConfig = cm.get_broker_config('IBKR')

for key, baseConfig in cm.instruments_data.items():
    if baseConfig['symbol'] in brokerConfig.instruments.keys() and baseConfig['type'] == 'futures':
        iconfig = cm.get_config('IBKR', key)
        t = ifera.load_data_tensor(instrument=iconfig, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)
    



In [ ]:
idata2 = ifera.InstrumentData(iconfig, device=torch.device('cuda:0'))

In [ ]:
ms = ifera.MarketSimulatorIntraday(idata2)

In [ ]:
import torch
import ifera
device=torch.device('cuda:0')

cm = ifera.ConfigManager()
iconfig = cm.get_config('IBKR', 'CL:1m')
dm = ifera.DataManager()
idata2 = dm.get_instrument_data(iconfig, device=device)
ms = ifera.MarketSimulatorIntraday(idata2)
t = (iconfig.liquid_start - iconfig.trading_start).total_seconds() // iconfig.time_step.total_seconds()

date_idx = torch.arange(0, idata2.data.shape[1], dtype=torch.int32, device=device)
time_idx = torch.full_like(date_idx, t, dtype=torch.int32, device=device)
position = torch.zeros_like(date_idx, dtype=torch.int32, device=device)
action = torch.full_like(date_idx, 1, dtype=torch.int32, device=device)
stop_loss = torch.full_like(date_idx, torch.nan, dtype=torch.float32, device=device)

%timeit ms.calculate_step(date_idx, time_idx, position, action, stop_loss)
#idata.data[0, 0:10]

In [ ]:
from einops import rearrange, repeat

parent_data = idata2.data
multiplier = 120
parent_steps = parent_data.shape[1]
padding = parent_steps - parent_steps // multiplier * multiplier
padding_data = torch.zeros((parent_data.shape[0], parent_data.shape[2]), dtype=parent_data.dtype, device=parent_data.device)
padding_data[:, 0:4] = parent_data[:, -1, 3, None]  # Open, High, Low, Close <- Last Close
padding_data = repeat(padding_data, "d c -> d g c", g=padding)
parent_data = torch.cat([parent_data, padding_data], dim=1)
rearrange(parent_data, "d (g n) c -> d g c n", n=120).shape

In [ ]:
idata2.instrument.contract_multiplier * (1051.0 - idata2.instrument.min_slippage - 1051.0) - 2.25, idata2.instrument.min_slippage * idata2.instrument.contract_multiplier

In [ ]:
import datetime as dt

iconfig = cm.get_config('IBKR', 'CL:1m')
t = ifera.load_data_tensor(instrument=iconfig, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)
t[-60 * 8 + 30], dt.date.fromordinal(int(t[-60 * 8 + 30, 0].item())), dt.timedelta(seconds=t[-60 * 8 + 30, 1].item())

In [ ]:
df = ifera.load_data(raw=True, instrument=iconfig, zipfile=True)

In [ ]:
# df[dt.datetime(2025, 1, 13, 9, 30)]
import pandas as pd

iconfig = cm.get_config('IBKR', 'NIY:1m')

df = ifera.load_data(raw=True, instrument=iconfig, zipfile=True)
t = ifera.load_data_tensor(instrument=iconfig, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)

print(df.loc[dt.datetime(2025, 1, 10, 9, 30)])
idx = -60 * 8 + 30 - 23 * 60 
s = dt.timedelta(seconds=t[idx, 1].item()).seconds
f"Date: {dt.date.fromordinal(int(t[idx, 0].item()))}, Time: {s//3600}:{(s%3600)//60}:{s%60:02d}, Open: {t[idx, 4].item():.2f}, High: {t[idx, 5].item():.2f}, Low: {t[idx, 6].item():.2f}, Close: {t[idx, 7].item():.2f}, Volume: {t[idx, 8].item():.0f}"



In [ ]:
df.tail()
df.loc[dt.datetime(2025, 1, 10, 9, 30)]

In [ ]:
import boto3

s3 = boto3.client('s3')

response = s3.list_objects(Bucket="kibotdata", Prefix='futures/1m/')
for obj in sorted(response.get('Contents', []), key=lambda x: x['Size'], reverse=True):
    print(obj['Key'], obj['Size'] // 1024**2)


In [ ]:
import ifera

cm = ifera.ConfigManager()
broker = cm.get_broker_config('IBKR')

for iconfig in broker.instruments:
    iconfig = cm.get_config('IBKR', f"{iconfig}:1m")
    df = ifera.load_data(raw=False, instrument=iconfig, zipfile=True)



In [ ]:
import ifera
import torch
from einops import rearrange

cm = ifera.ConfigManager()
broker = cm.get_broker_config('IBKR')

for key in cm.instruments_data.keys():
    base_inst = cm.get_base_instrument_config(key)
    if base_inst.symbol not in broker.instruments:
        continue
    iconfig = cm.get_config('IBKR', key)
    t = ifera.load_data_tensor(instrument=iconfig, reset=False, device=torch.device('cuda:0'), dtype=torch.float32)
    t = rearrange(t, '(d t) c -> d t c', t = iconfig.total_steps)
    print(iconfig.symbol, t.shape)
    break

In [ ]:
data = t[:, :, 4:]

In [ ]:
import torch
import ifera
channels = {"Open":0, "High":1, "Low":2, "Close":3, "Volume":4}

# Relative True Range: max(high, prev_close) / min(low, prev_close), and simple high/low ratio for the first bar
rtr = torch.zeros_like(data[:, :, channels["Close"]])
rtr[:, 0] = data[:, 0, channels["High"]] / data[:, 0, channels["Low"]] - 1.0
rtr[:, 1:] = torch.max(data[:, 1:, channels["High"]], data[:, :-1, channels["Close"]]) / torch.min(data[:, 1:, channels["Low"]], data[:, :-1, channels["Close"]]) - 1.0

# Where volume is zero set rtr to be the same as the previous value

In [ ]:
from einops import rearrange, repeat

channels = {"Open":0, "High":1, "Low":2, "Close":3, "Volume":4}

volume = data[:, :, channels["Volume"]].to(torch.int32)

# Find first non-zero volume each day
first_non_zero = torch.argmax((volume > 0).to(torch.int8), dim=1)

rtr = torch.zeros_like(data[:, :, channels["Close"]])
rtr[:, first_non_zero] = data[:, first_non_zero, channels["High"]] / data[:, first_non_zero, channels["Low"]] - 1.0

mask = volume != 0
vdata = data[mask, :]
vrtr = rtr[mask]

raw_rtr = torch.max(vdata[1:, channels["High"]], vdata[:-1, channels["Close"]]) / torch.min(vdata[1:, channels["Low"]], vdata[:-1, channels["Close"]]) - 1.0
vrtr[1:] = torch.where(vrtr[1:] == 0, raw_rtr, vrtr[1:])

artr = ifera.ema_slow(vrtr, 1/14)

artr

In [ ]:
import torch

t = torch.tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]], dtype=torch.float32)
window = 3

n = min(t.shape[-1], window)
tail = t.unfold(dimension=-1, size=n, step=1).mean(dim=-1)

head_nom = torch.cumsum(t[..., :n-1], dim=-1)
head_denom = torch.arange(1, n, device=t.device, dtype=t.dtype)
head = head_nom / head_denom

sma = torch.cat([head, tail], dim=-1)
sma


In [ ]:
import torch
import ifera
from einops import rearrange

cm = ifera.ConfigManager()
iconfig = cm.get_config("IBKR", "CL:1m")

dm = ifera.DataManager()
idata = dm.get_instrument_data(iconfig)

batch_size = idata.data.shape[0]-250

openPolicy = ifera.AlwaysOpenPolicy(direction=1)
initStopPolicy = ifera.InitialArtrStopLossPolicy(
    instrument_data=idata, atr_multiple=3.0
)
maintenancePolicy = ifera.ScaledArtrMaintenancePolicy(
    instrument_data=idata,
    stages=["1m", "5m", "15m", "1h", "4h", "1d"],
    atr_multiple=3.0,
    wait_for_breakeven=False,
    minimum_improvement=0.0,
)

tradingPolicy = ifera.TradingPolicy(
    instrument_data=idata,
    open_position_policy=openPolicy,
    initial_stop_loss_policy=initStopPolicy,
    position_maintenance_policy=maintenancePolicy,
)

#tradingPolicy = torch.compile(tradingPolicy)

ms = ifera.MarketSimulatorIntraday(instrument_data=idata)

t = (iconfig.liquid_start - iconfig.trading_start).total_seconds() // iconfig.time_step.total_seconds() - 1

date_idx = torch.arange(0, batch_size, dtype=torch.int32, device=idata.device)
time_idx = torch.full_like(date_idx, t, dtype=torch.int32, device=idata.device)
position = torch.zeros_like(date_idx, dtype=torch.int32, device=idata.device)
prev_stop_loss = torch.full_like(date_idx, torch.nan, dtype=torch.float32, device=idata.device)
execution_price = torch.full_like(date_idx, torch.nan, dtype=torch.float32, device=idata.device)

total_profit = torch.zeros_like(date_idx, dtype=torch.float32, device=idata.device)

# Open position
action, stop_loss = tradingPolicy(date_idx, time_idx, position, prev_stop_loss, execution_price)
time_idx += 1
profit, position, execution_price, _ = ms.calculate_step(date_idx, time_idx, position, action, stop_loss)
position = position.clone()
total_profit += profit

while (position != 0).any():
    prev_stop_loss = stop_loss.clone()
    action, stop_loss = tradingPolicy(date_idx, time_idx, position, prev_stop_loss, execution_price)
    date_idx, time_idx = idata.get_next_indices(date_idx, time_idx)
    action = torch.where((date_idx == idata.data.shape[0] - 1) & (time_idx == idata.data.shape[1] - 1), -position, 0)
    profit, position, execution_price, _ = ms.calculate_step(date_idx, time_idx, position, action, stop_loss)
    position = position.clone()
    total_profit += profit

max_idx = total_profit.argmax()
print(f"Max index: {max_idx}, Total profit: {total_profit[max_idx].item():.4f}, date_idx: {date_idx[max_idx].item()}, time_idx: {time_idx[max_idx].item()}")


In [ ]:
import torch
import numpy as np

t = torch.arange(0, 10, dtype=torch.int32)
mask = torch.zeros_like(t, dtype=torch.bool)

t, mask, torch.cat((t[mask], torch.tensor([np.iinfo(np.int32).max], dtype=torch.int32)), dim=0).min().item(), t[t< 5]



In [1]:
import ifera

instrument = ifera.ConfigManager().get_config("IBKR", "TN:1m")
url = ifera.make_instrument_url("file", "processed", instrument)
print(url)
print(ifera.FileOperations.exists(url))

#print(ifera.FileOperations.exists("s3://ifera-marketdata/processed/futures/1m/CL.zip"))
#print(ifera.FileOperations.exists("github:iferaorg/iferadata/data/instruments.json"))
#print(ifera.FileOperations.exists("file:data/processed/futures/1m/CL.zip"))

df = ifera.load_data(raw=False, instrument=instrument)

print(ifera.FileOperations.exists(url))

file:data/processed/futures/1m/TN.zip
True


Loading data from data/raw/futures/1m/TN.zip: 100%|██████████| 2505060/2505060 [00:00<00:00, -3287639.39lines/s]


Converting datetime columns...
Processing groups...


Processing trade dates: 100%|██████████| 2393/2393 [00:24<00:00, 98.53it/s] 


Performing final calculations...
Saving processed data...
Processed data saved to data/raw/futures/1m/TN.zip


Uploading processed/futures/1m/TN.zip: 100%|██████████| 31.6M/31.6M [00:08<00:00, 3.90MB/s]
Loading data from data/processed/futures/1m/TN.zip: 100%|██████████| 3302340/3302340 [00:03<00:00, 1099909.28lines/s]

True


In [ ]:
from pathlib import Path

Path()

PosixPath('.')